In [1]:
!pip install openai==0.28
!pip install openai textblob
!pip install gradio

In [2]:
# Import libraries
import openai
from textblob import TextBlob

In [3]:
# Set your OpenAI API key
openai.api_key = "API KEY"  # Replace with your actual OpenAI API key

In [4]:
# Function to analyze sentiment
def analyze_sentiment(text):
    """
    Analyze the sentiment of the input text using TextBlob.
    Returns: 'positive', 'negative', or 'neutral'
    """
    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity
    if sentiment > 0.1:
        return "positive"
    elif sentiment < -0.1:
        return "negative"
    else:
        return "neutral"

In [5]:
# Function to detect crisis-related keywords
def detect_crisis(input_text):
    """
    Detect if the input contains crisis-related keywords.
    Returns: True if a crisis is detected, else False
    """
    crisis_keywords = ["suicide", "harm myself", "end my life", "kill myself"]
    for keyword in crisis_keywords:
        if keyword in input_text.lower():
            return True
    return False

In [6]:
# ChatGPT interaction function
def chat_with_gpt(prompt):
    """
    Interact with OpenAI's ChatGPT API to get a response based on the input prompt.
    """
    try:
        # Call the ChatGPT API
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  # Model to use
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are acting as a therapist for the user. The user wants to talk about how they are feeling "
                        "right now and is not necessarily looking for solutions to their problems. Focus on listening, "
                        "understanding, and reflecting back what the user expresses. Respond empathetically and like a "
                        "real person, without giving generic or robotic answers. If user asks you to help them then tell what they need to hear"
                    )
                },
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7,
        )
        # Extract and return the response
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Error: {e}"

In [7]:
# Function to integrate sentiment and ChatGPT
def chat_with_gpt_with_sentiment(prompt):
    """
    Enhance the ChatGPT response by analyzing the user's sentiment and including it in the prompt.
    """
    sentiment = analyze_sentiment(prompt)
    sentiment_prefix = f"The user seems to be feeling {sentiment}. " if sentiment != "neutral" else ""
    prompt = sentiment_prefix + prompt
    return chat_with_gpt(prompt)

In [8]:
# Main chat loop
def run_chatbot():
    print("Welcome to the Mental Health Chatbot! Type 'exit' to end the chat.\n")

    while True:
        # Take user input
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Take care! Remember, you are not alone.")
            break

        # Check for crisis-related keywords
        if detect_crisis(user_input):
            print("Chatbot: I'm sorry you're feeling this way. Please reach out to a trusted person or call a crisis hotline immediately. You are not alone.")
            continue

        # Get the chatbot's response with sentiment analysis
        bot_response = chat_with_gpt_with_sentiment(user_input)
        print(f"Chatbot: {bot_response}")

# Run the chatbot
#run_chatbot()

In [9]:
# Import Gradio
import gradio as gr

# Chatbot function with chat history
def chatbot_response(user_input, history):
    """
    Generate a response from the chatbot while maintaining the chat history.
    """
    # Add the user's input to the history
    history = history or []
    history.append(("User", user_input))

    # Check for crisis-related keywords
    if detect_crisis(user_input):
        bot_response = (
            "I'm sorry you're feeling this way. Please reach out to a trusted person or call a crisis hotline immediately. You are not alone."
        )
    else:
        # Get the chatbot's response with sentiment analysis
        bot_response = chat_with_gpt_with_sentiment(user_input)

    # Add the bot's response to the history
    history.append(("Therapist", bot_response))
    return history, history  # Return updated history for display

In [10]:
# Define the Gradio interface
interface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        "text",  # User input
        "state"  # Chat history
    ],
    outputs=[
        "chatbot",  # Chat history display
        "state"     # Updated chat history state
    ],
    title="Mental Health Chatbot",
    description="Talk to a therapist about how you're feeling.",
    theme="compact"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1024: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-674b9ddc-260320612422af271c1eb157;f3bb883e-64d2-44e9-81d6-401d97c7dbce)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: Us

In [11]:
# Launch the interface
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://10a3ed40cb19487fdb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
